# 1. Read JSON file

In [43]:
import json

#Read JSON data into the datastore variable
filename = 'online course data.json'
with open(filename, 'r', encoding='UTF-8') as f:
    onlineCourses_JSONarr = json.load(f)

#Use the new datastore datastructure
print(len(onlineCourses_JSONarr))
print(onlineCourses_JSONarr[0].keys())

826
dict_keys(['cTitle', 'cProvider', 'cPhotoLink', 'cDirectLink', 'cDescription', 'cWeekSection', 'cAuthor', 'cLanguage', 'tmpCateg', 'tmpId'])


In [154]:
for i in range(0,10):
    print(onlineCourses_JSONarr[i]['cTitle'])

Machine Learning
用Python玩转数据 Data Processing Using Python
Building a Data Science Team
Command Line Tools for Genomic Data Science
A Crash Course in Data Science
Data Science in Real Life
Materials Data Sciences and Informatics
Computer Science:  Algorithms, Theory, and Machines
Data Science Methodology
Data Science Ethics


In [446]:
str1=str(onlineCourses_JSONarr[114]['cWeekSection'])
str1=re.sub('\n\n',' ',str1)
str1=re.sub('\n',' ',str1)
str1

"[{'name': 'Data Warehouse Concepts and Architectures', 'description': 'Module 1 introduces the course and covers concepts that provide a context for the remainder of this course. In the first two lessons, you’ll understand the objectives for the course and know what topics and assignments to expect. In the remaining lessons, you will learn about historical reasons for development of data warehouse technology, learning effects, business architectures, maturity models, project management issues, market trends, and employment opportunities. This informational module will ensure that you have the background for success in later modules that emphasize details and hands-on skills.You should also read about the software requirements in the lesson at the end of module 1. I recommend that you try to install the software this week before assignments begin in week 2.', 'timeCommitment': 13813000, 'id': 'HfGNt', 'lessonIds': ['Qrhdz', 'N9fJu', '92FTQ', 'q4vBJ', 'xJgms', 'gX8XP', 'akxlZ', 'aNJaW',

# 2. Helper functions

In [393]:
def addProfessor(prof,courseId):
    '''Check if professor exists in db, creates new professor if not and the TEACHES relationship with course'''
    query = 'MERGE (professor:Professor {name: \"' + prof + '\"})' +\
            'MERGE (course:Course {tmpId: \"' + courseId + '\"})' +\
            'CREATE UNIQUE (professor)-[:TEACHES]->(course)'
    graph.run(query)


In [394]:
def addLanguage(lang ,courseId):
    '''Check if professor exists in db, creates new language if not and the LANGUAGE relationship with course'''
    query = 'MERGE (language:Language {name: \"' + lang + '\"})' +\
            'MERGE (course:Course {tmpId: \"' + courseId + '\"})' +\
            'CREATE UNIQUE (course)-[:LANGUAGE]->(language)'
    graph.run(query)
                

In [395]:
def addCategory(cat ,courseId):
    '''Check if professor exists in db, creates new category if not and the category relationship with course'''
    query = 'MERGE (category:Category {number: \"' + cat + '\"})' +\
            'MERGE (course:Course {tmpId: \"' + courseId + '\"})' +\
            'CREATE UNIQUE (course)-[:IS_CATEGORY]->(category)'
    graph.run(query)
                

In [396]:
def addProvider(prov ,courseId):
    '''Check if professor exists in db, creates new provider if not and the provider relationship with course'''
    query = 'MERGE (provider:Provider {number: \"' + prov + '\"})' +\
            'MERGE (course:Course {tmpId: \"' + courseId + '\"})' +\
            'CREATE UNIQUE (provider)-[:PROVIDES]->(course)'
    graph.run(query)
                

# 3. Connect to Neo4j DB

In [433]:
#connect and clean db
from py2neo import Graph

graph = Graph("http://localhost:7474", user='EvaArevalo', password='yhwh123')
graph.delete_all()

In [434]:
#Create uniqueness constraints
graph.run('CREATE CONSTRAINT ON (professor:Professor) ASSERT professor.name IS UNIQUE')
graph.run('CREATE CONSTRAINT ON (course:Course) ASSERT course.tmpId IS UNIQUE')
graph.run('CREATE CONSTRAINT ON (language:Language) ASSERT language.language IS UNIQUE')
graph.run('CREATE CONSTRAINT ON (category:Category) ASSERT category.number IS UNIQUE')
graph.run('CREATE CONSTRAINT ON (provider:Provider) ASSERT provider.name IS UNIQUE')

In [435]:
#drop constraints if needed
#graph.run('DROP CONSTRAINT ON (course:Course) ASSERT course.title IS UNIQUE')

In [447]:
#Create Nodes and relationships
import re

for course in onlineCourses_JSONarr:
    #Create Course node
    description = ''
    weekSection = ''
    photoLink = ''
    directLink = ''
    if 'cPhotoLink' in course and course['cPhotoLink']!=None:
        photoLink = course['cPhotoLink']
    if 'cDirectLink' in course and course['cDirectLink']!=None:
        directLink = course['cDirectLink']
    if 'cDescription' in course and course['cDescription']!=None:
        description = re.sub('\"', '\'', course['cDescription'])
        description = re.sub('\n\n', ' ', description)
        description = re.sub('\n', ' ', description)
    if 'cWeekSection' in course and course['cWeekSection']!=None:
        weekSection = re.sub('\"', '\'', str(course['cWeekSection']))
        weekSection = re.sub('\n\n', ' ', weekSection)
        weekSection = re.sub('\n', ' ', weekSection)
    title = re.sub('\'', ',', course['cTitle'])
    query = 'CREATE (course:Course {tmpId:\"' + str(course['tmpId']) + '\", ' +\
                                    'title:\'' + title + '\', ' +\
                                    'photoLink:\"' + photoLink + '\", ' +\
                                    'directLink:\"' + directLink + '\", ' +\
                                    'description:\"' + description +  '\", ' +\
                                    'weekSection:\"' + weekSection + '\"})' 

    graph.run(query)
    #  'weekSection:\"' + weekSection + '\", ' +\
    
    #Add relationships and other nodes
    addLanguage(course['cLanguage'], str(course['tmpId']))
    addProvider(course['cProvider'], str(course['tmpId']))
    for cat in course['tmpCateg']:
        addCategory(str(cat),str(course['tmpId']))
    for prof in course['cAuthor']:
        profname = re.sub('\"',',',prof)
        addProfessor(profname, str(course['tmpId']))


# 3. Draw graph

In [ ]:
import scripts.vis
from scripts.vis import draw

options = {"Professor": "name", "Course": "title", "Language":"language", "Category":"number"}
draw(graph, options)